# Calculating MS5351M PLL Settings


There are several parameters that we can modify to choose the output frequency. The constraint that we want quadrature I/Q outputs as part of CLK0 and CLK1 provides a strong restriction on possible frequencies. We only use PLLA for the quadrature output, as the frequencies are identical.

The constraints are:
* Quadrature requires an even multiple, $N$, of the output frequency to be the PLL frequency. The delay register (CLKx_PHOFF) must then be set to $N$, to get the quadrature output on the second clock.
* The PLL frequency must be between 600 and 900 MHz
* THe input clock is $f_{clk} = 25$ MHz.
* Above 150 MHz, the output divide ratio, $M_{output} R$, must be 4.
		
The output frequency is given by the expression
$$ f_{vco} = (f_{clk} / D) * M_{feedback} $$
$$ f_{out} = \frac{f_{vco}}{M_{output} R} $$


$$ M_{feedback} = a + b/c $$
where a, b, c are roughly integers

P1[17:0] = (a << 7) + 



In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import math

def multisynth(f_clk, a, b, c, d):
    return (f_clk/d) * (a + b/c)


def get_closest_vco(f_clk, vco_previous, vco_desired):
    best = {}
    best['fit'] = 9e99

    for d in [1, 2, 4, 8]:
        for a in range(1, 90):
            for b in range(0, 90):
                for c in range(1, 5000):
                    
                    m = a + b/c
                    if m < 15:
                        break
                    if m > 90:
                        break
                        
                    f_vco = multisynth(f_clk, a, b, c, d)
                    df = f_vco - vco_desired
                    fit = math.sqrt(df*df)
                    if fit < best['fit']:
                        best['fit'] = fit
                        best['a'] = a
                        best['b'] = b
                        best['c'] = c
                        best['d'] = d
                        best['f_vco'] = f_vco
                        best['df'] = df
                    if fit < 1.0:
                        return best
    print(best)            
    return best
        
def synth(f_clk, f_desired):
    ret = {}
    
    if f_desired > 150e6:
        n_start = 2
        n_stop = 2
        N = 4
    else:
        # Search for best f_vco (an even integer multiple)        
        n_stop = math.floor(900e6 / (2*f_desired))+1
        n_start = math.floor(600e6 / (2*f_desired))
                
    best = {}
    best['fit'] = 9e99
        
    for n in range(n_start, n_stop+1):
        N = 2*n # Use even multipliers only for quadrature
        f_vco = f_desired*N
        if (f_vco > 600e6) and (f_vco < 900e6):
            # Now try and get f_vco from f_in
                
            test = get_closest_vco(f_clk, f_vco, f_vco)
                
            if test['fit'] < best['fit']:
                best = test
                    
            if best['fit'] < 1:
                break
    ret['multisynth'] = best
    ret['f'] = best['f_vco']/N
    ret['N'] = N

    
    return ret

In [3]:
synth(25e6, 13580000)

{'fit': 512.8205128908157, 'a': 49, 'b': 38, 'c': 39, 'd': 2, 'f_vco': 624679487.1794871, 'df': -512.8205128908157}


{'multisynth': {'fit': 0.0,
  'a': 26,
  'b': 46,
  'c': 625,
  'd': 1,
  'f_vco': 651840000.0,
  'df': 0.0},
 'f': 13580000.0,
 'N': 48}

In [4]:
freq = np.random.uniform(low=1e6, high=150e6, size=1000)
data = []

if False:
    for f in freq:
        x = synth(25e6, f)
        print(f, x)
        print(x.keys())
        data.append(x['multisynth']['df'])
        plt.hist(data, bins='fd')

## Band Plans

Above shows that we can do quadrature synthesis within about 10 Hz of most frequencies. This brute force approach does require a bit too much computation, so we come up with a band plan, over a limited range of frequencies. The basic idea is to use a low VCO frequency at the low end of the band, and to change only one of the synthesizer numbers (either b or c). So a band has a fixed choice of the output divider (for quadrature), and changes only the VCO frequency using a fixed $a$ and $b$

The idea is to choose a delta_f for the band as well as f_low and f_high

In [5]:
import sympy as sp

f_low = sp.Symbol('f_low')
f_high = sp.Symbol('f_high')
N = sp.Symbol('N')
f_clk = sp.Symbol('f_clk')

a = sp.Symbol('a')
c = sp.Symbol('c')
b = sp.Symbol('b')
f_vco = f_clk * (a + b/c)
f = f_vco / N
f1 = f.subs(b,b+1)
f0 = f.subs(b,b)
f

f_clk*(a + b/c)/N

In [6]:
delta_f = sp.simplify(f1 - f)
delta_f

f_clk/(N*c)

In [7]:
f0

f_clk*(a + b/c)/N

So step 1 is to figure out $N$ which is the even clock output multiplier. This is done by using $f_{low}$ and setting the $f_{vco}$ to be just above 600 MHz.

$$ N = 2 ceil \left( \frac{600,000,000}{ 2 f_{low}} \right) $$

Then, once N is established, use f_low to calculate $a$
$$ a = N* \frac{f_{low}}{f_{clk}} $$

Now work out $c$ from the expression for delta_f.

## A Band Class

Lets us do all the necessary setup.

In [34]:
class Band:
    def __init__(self, f_clk, f_low, delta_f, f_high):
        self.f_clk = f_clk
        self.f_low = f_low
        self.delta_f = delta_f
        self.f_high = f_high

        self.N = 2*np.ceil(600000000 / (2*self.f_low) )
        self.a =  np.floor(self.N * self.f_low / self.f_clk)
        self.c = np.ceil(self.f_clk / (self.N * self.delta_f))

        a_frac = (self.N * self.f_low / self.f_clk) - self.a
        self.b0 = np.floor(a_frac*self.c)

    def get_actual_delta_f(self):
        return self.f_clk / (self.N*self.c)
    
    def f(self, b):
        f_vco = self.f_clk*(self.a + ((self.b0 + b) / self.c))
        return f_vco / self.N
    
    def __repr__(self):
        return f"a: {self.a}, N: {self.N}, b0: {self.b0} c: {self.c}, df: {self.get_actual_delta_f()}, vco_lo={self.f_clk*self.a}"

In [35]:
b6 = Band(f_clk=25e6, f_low=50e6, delta_f=10, f_high=51e6)
b6

a: 24.0, N: 12.0, b0: 0.0 c: 208334.0, df: 9.999968000102399, vco_lo=600000000.0

In [36]:
b160 = Band(f_clk=25e6, f_low=1.8e6, delta_f=10, f_high=1.95e6)
b160

a: 24.0, N: 334.0, b0: 359.0 c: 7486.0, df: 9.998704167939835, vco_lo=600000000.0

In [37]:
b80 = Band(f_clk=25e6, f_low=3.5e6, delta_f=10, f_high=3.9e6)
b80

a: 24.0, N: 172.0, b0: 1162.0 c: 14535.0, df: 9.999920000639994, vco_lo=600000000.0

In [38]:
b40 = Band(f_clk=25e6, f_low=7.00e6, delta_f=10, f_high=7.3e6)
b40

a: 24.0, N: 86.0, b0: 2325.0 c: 29070.0, df: 9.999920000639994, vco_lo=600000000.0

In [39]:
b20 = Band(f_clk=25e6, f_low=14e6, delta_f=10, f_high=14.35e6)
b20

a: 24.0, N: 44.0, b0: 36364.0 c: 56819.0, df: 9.999856002073571, vco_lo=600000000.0

In [40]:
b10 = Band(f_clk=25e6, f_low=28e6, delta_f=10, f_high=29.7e6)
b10

a: 24.0, N: 22.0, b0: 72727.0 c: 113637.0, df: 9.999944000313599, vco_lo=600000000.0

In [41]:
b2 = Band(f_clk=25e6, f_low=144e6, delta_f=10, f_high=148e6)
b2

a: 34.0, N: 6.0, b0: 233333.0 c: 416667.0, df: 9.9999920000064, vco_lo=850000000.0

In [46]:
b2.f(3)

144000024.79998013

In [43]:
b6.f(0)

50000000.0